# NSSC 2 
# Exercise 1: MPI-Parallelization of a Jacobi Solver

Group No. 12

Aman Bhardwaj (12333472)

Florian Frech (12308544)


## Task 1: Questions

### 1. Describe the advantages / disadvantages of a two-dimensional decomposition 
(compared to a one dimensional decomposition)

Advantages of a 2D decomposition:

- It is easier to handle complex geometries which are irregular or intricate in shape as it decomposes the domain into smaller subdomains that can be handled and distributed evenly across individual processes.

- It provides clear representation of discontinuous properties and interfaces, thus making it easier to apply  arious numerical methods and boundary conditions leading to more accurate simulations.

- In a 2-D decomposition, each region can independently control and generate its own mesh, that provides more efficient mesh generation.

- The neighbouring grid points close to each other are assigned the same MPI process which helps in better cache utilization and more data locality.

- Conservative methods of physical quantities can be employed because they can be applied to individual regions independently.

Disadvantages of a 2D decomposition:

- Due to less connectivity of information between the neighbouring processes, it can become more challenging to implement and handle communication between neighbouring processes, which in turn lead to more complexity and potential errors.

- In some cases when the grid size is large in 2-D decomposition, it can result in increased communication volume as the information is to be exchanged in both dimensions.

- Storing stencil coefficients for each control volume within each MPI process can lead to increased memory usage, when the grid size is large and reduce scalability.

### 2. Discuss if the decomposition of the domain changes the order of computations performed during a single jacobi iteration
(i.e., if you expect a numerically identical result after each iteration, or not)

In a Domain decomposition, the order of computations within a single Jacobi iteration does not change and remains consistent as each MPI process independently updates the values of grid points within the assigned sub-domain which are based on the values of previous iteration from the neighbouring grid
points.

Also, because the computations proceed sequentially within each segment and subdomain, each grid point is  updated one after the other in the decompositions leading to numerically identical result.

### 3. An extension to the ghost layer would be to use a wider layer (of more than one ghost cell). This allows to perform multiple independent iterations before a communication of the ghost layer has to take happen.
Comment in which situation (w.r.t. the available bandwidth or latency between MPI-processes) multiple independent iterations are potentially advantageous.

Using a wider layer to perform multiple independent iterations before setting up a communication of the ghost layers can be advantageous in many ways in the cases when the bandwidth between MPI processes is high, domain is large and  the latency is low, so this decreases the time spent on communication compared to that of computation.

It will result in better utilization of the resources and improved performance of the Jacobi solver. Also, all processes can proceed in a similar pace and reduce communication delays ensuring more balanced progress across the entire domain.

### 4. How big is the sum of all L2 caches for 2 nodes of the IUE-cluster

The nodes of the infrastructure are equipped with two Intel Xeon Gold 6248 processors at 2.5 GHz with 20 cores and 40 available threads.

The global L2 cache totals 20 MB with 1 MB per processor. The cummulative L2 cache across both nodes in the IUE Cluster is 80 MB.

## Task 2: One-Dimensional Decomposition

### Parallel Speed Up and Efficiency

The parallel speed up is measured as the ratio of the time taken to execute the task sequentially ($T_{sequentially}$) to the time taken to execute in in parallel ($T_{parallel}$):

$$
S = \frac{T_{sequential}}{T_{parallel}}
$$

In the ideal case, the spedup behaves proportionally to the number of used processes, means with doubled number of processes we would solve the problem in half of the original time.

However, by Amdahl's Law we know, there is an upper bound that can be achieved by parallelizing a computation. It states that the overall fraction is limited by the fraction of the program that cannot be parallelized.

Amdahl's Law:
$$
S_{max} = \frac{1}{(1-P) + \frac{P}{n_p}}
$$

 - $S_{max}$: Maximum speedup
 - $P$: Fraction that cannot be parallelized
 - $N$: Number of processes / threads

Efficiency in parallel computing is measured as the ratio of the speedup achieved by parallel execution to the number of processes / threads. The efficiency describes how well the parallel algorithm scales with increasing resources.

$$
E = \frac{S}{n_p}
$$

Similar to Amdahl's Law, an efficiency of $E=1$ would be ideal and means, the used resources are optimally utilized and the speedup is directly proportional to the number of processing units.



<center>
    <figure>
        <img src ="speedup.png" alt="" width="1000" />
        <figcaption>Fig. 1: Speed up of total runtime</figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src ="efficiency.png" alt="" width="1000" />
        <figcaption>Fig. 2: Efficiency</figcaption>
    </figure>
</center

Figure 1 and figure 2 show the speed and the efficiency for a Jacobi solver with a 1D grid decomposition for different grid sizes $N²$.

A range between 1 and 40 processes were used on the cluster to perform a benchmark test.

The plots in both figures shows for all problem sizes spikes in their curves. The reason for this might be the decomposition of the grid.

The grid is decomposed in a way, that the last process in the cartesian MPI topology takes care of the remainder rows.
Thus in some test scenarios (pair of number of processes and problem size N), there is a significant difference in the number of rows which are processed by the non-remainder processes and the remainder process.
For example, in the case $N=1000$ and $n_p=25$, where no rmainders are required, a remarkable speed up can be identified compared to the case $N=1000$ with $p=24$.

In general, the speed up is more increased for larger parblem sizes since there more computations per communications can be done than for smaller problem sizes.
The efficiency graphs in figure 2 supports this fact, as for a great number of processes and a small problem size is significantly lower than for larger problem sizes.

For $n_p$ up to 10, the speed up for $N = 2000$ is almost ideal since almost linear.

For the communication non-blocking sends and recieves are used without derived datatypes. The subgrids are implemented as row-major std::vector<double>(subgrid_size) with northern and southern ghost layers.

To conclude, the 1D decomposition depends on how the rows of the grids can be split bewteen the processes and if unfavourable remainders exist. The remainder problem can be improved if the remainder rows are distrubuted most optimally over all processes. However, this increases the complexity of the code and the handling of the subgrid sizes.